In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time

import numpy as np
from multiprocess import Pool

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

%matplotlib inline

Setting environment for Precision WorkStation


No vtk
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
atlas_spec = dict(name='atlasV6',
                   vol_type='score'    ,               
                    resolution='10.0um'
                   )

In [3]:
atlas_structures_wrt_canonicalAtlasSpace_atlasResol = \
DataManager.load_original_volume_all_known_structures_v3(atlas_spec, in_bbox_wrt='canonicalAtlasSpace',
                                                        out_bbox_wrt='canonicalAtlasSpace')

Prior structure/index map not found. Generating a new one.


In [5]:
stack = 'CHATM2'

In [6]:
T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = bp.unpack_ndarray_file('/home/yuncong/' + stack + '_T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol.bp')

In [7]:
registered_atlas_structures_wrt_wholebrainWithMargin_atlasResol = \
{name_s: transform_volume_v4(volume=vo, transform=T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol, return_origin_instead_of_bbox=True)
for name_s, vo in atlas_structures_wrt_canonicalAtlasSpace_atlasResol.iteritems()}

/home/yuncong/Brain/utilities/registration_utilities.py:1561: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(volume_m_aligned_to_f.dtype, np.float):
Interpolating/filling sparse volume: 0.07 seconds.
Interpolating/filling sparse volume: 0.33 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.06 seconds.
Interpolating/filling sparse volume: 0.02 seconds.
Interpolating/filling sparse volume: 0.13 seconds.
Interpolating/filling sparse volume: 0.13 seconds.
Interpolating/filling sparse volume: 0.06 seconds.
Interpolating/filling sparse volume: 0.02 seconds.
Interpolating/filling sparse volume: 0.20 seconds.
Interpolating/filling sparse volume: 0.33 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.07 seconds.
Interpolating/filling sparse volume: 

In [51]:
# Define volume ROI for which to compute scores based on outline registered atlas.

In [8]:
registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol = \
{name_s: (o[0], o[0] + v.shape[1] - 1, o[1], o[1] + v.shape[0] - 1, o[2], o[2] + v.shape[2] - 1)
 for name_s, (v, o) in registered_atlas_structures_wrt_wholebrainWithMargin_atlasResol.iteritems()}

In [9]:
registered_atlas_structures_xyzTwoCorners_wrt_wholebrainWithMargin_atlasResol = \
{name_s: ((o[0], o[2], o[4]), (o[1], o[3], o[5]))
for name_s, o in registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol.iteritems()}

In [10]:
registered_atlas_structures_xyzCorners_wrt_wholebrainWithMargin_atlasResol = \
{name_s: ((o[0], o[2], o[4]), (o[0], o[2], o[5]), (o[0], o[3], o[4]), (o[0], o[3], o[5]), \
         (o[1], o[2], o[4]), (o[1], o[2], o[5]), (o[1], o[3], o[4]), (o[1], o[3], o[5]))
for name_s, o in registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol.iteritems()}

In [11]:
from data_manager import CoordinatesConverter

In [12]:
converter = CoordinatesConverter(stack=stack)

In [13]:
# registered_atlas_structures_wrt_wholebrainXYcropped_xysec = {}

# for name_s, corners_xyz in registered_atlas_structures_xyzCorners_wrt_wholebrainWithMargin_atlasResol.iteritems():
# #     print name_s
#     registered_atlas_structures_wrt_wholebrainXYcropped_xysec[name_s] = \
#     converter.convert_frame_and_resolution(p=corners_xyz, 
#                                        in_wrt=('wholebrainWithMargin', 'sagittal'),
#                                       in_resolution='10.0um',
#                                       out_wrt=('wholebrainXYcropped', 'sagittal'),
#                                       out_resolution='image_image_section').astype(np.int)

In [14]:
registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners = {}

for name_s, corners_xyz in registered_atlas_structures_xyzTwoCorners_wrt_wholebrainWithMargin_atlasResol.iteritems():
#     print name_s
    registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[name_s] = \
    converter.convert_frame_and_resolution(p=corners_xyz, 
                                       in_wrt=('wholebrainWithMargin', 'sagittal'),
                                      in_resolution='10.0um',
                                      out_wrt=('wholebrainXYcropped', 'sagittal'),
                                      out_resolution='image_image_section').astype(np.int)

In [15]:
registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners

{'10N_L': array([[15257, 10140,   152],
        [19534, 12540,   178]]), '10N_R': array([[15257, 10140,   181],
        [19534, 12540,   207]]), '12N': array([[15104, 10509,   158],
        [19196, 12724,   200]]), '3N_L': array([[4950, 7032,  165],
        [6980, 8263,  179]]), '3N_R': array([[4950, 7032,  180],
        [6980, 8263,  194]]), '4N_L': array([[6827, 7309,  162],
        [7473, 8047,  172]]), '4N_R': array([[6827, 7309,  187],
        [7473, 8047,  197]]), '5N_L': array([[ 8057,  9186,    96],
        [10519, 12048,   125]]), '5N_R': array([[ 8057,  9186,   234],
        [10519, 12048,   263]]), '6N_L': array([[10734, 10232,   155],
        [11596, 11063,   167]]), '6N_R': array([[10734, 10232,   192],
        [11596, 11063,   204]]), '7N_L': array([[10519, 12571,    98],
        [13750, 15217,   145]]), '7N_R': array([[10519, 12571,   214],
        [13750, 15217,   261]]), '7n_L': array([[ 9104,  9955,    90],
        [12180, 13678,   165]]), '7n_R': array([[ 9104,  9955

In [16]:
save_json(registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners, 
          '/home/yuncong/' + stack + '_registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners.json')

In [15]:
# registered_atlas_structures_wrt_wholebrainWithMargin_section_limits = {}

# for name_s, b in registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol.iteritems():
# #     print name_s
#     registered_atlas_structures_wrt_wholebrainWithMargin_section_limits[name_s] = \
#     converter.convert_frame_and_resolution(p=[[np.nan, np.nan, b[4]], [np.nan, np.nan, b[5]]], 
#                                        in_wrt=('wholebrainWithMargin', 'sagittal'),
#                                       in_resolution='10.0um',
#                                       out_wrt='wholebrain',
#                                       out_resolution='image_image_section')[:,2].astype(np.int)

In [35]:
# registered_atlas_structures_wrt_wholebrainWithMargin_section_limits

In [36]:
# save_json(registered_atlas_structures_wrt_wholebrainWithMargin_section_limits, 
#           '/home/yuncong/' + stack + '_structure_section_limits.json')

In [17]:
from learning_utilities import *

In [18]:
batch_size = 256
model_dir_name = 'inception-bn-blue'
model_name = 'inception-bn-blue'
model, mean_img = load_mxnet_model(model_dir_name=model_dir_name, model_name=model_name, 
                                   num_gpus=1, batch_size=batch_size)

/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:53: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:65: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [19]:
# out_resolution_um = 10.
# out_resolution_str = '%.1fum' % out_resolution_um

output_resolution = '10.0um'
out_resolution_um = convert_resolution_string_to_um(resolution=output_resolution, stack=stack)

# detector_id = 998
detector_id = 799
# print 'input_version:', detector_settings.loc[detector_id]['input_version']

In [20]:
detector_setting = {}
detector_setting['feature_classifier_id'] = 899
detector_setting['windowing_id'] = 7

In [40]:
clfs = DataManager.load_classifiers(classifier_id=detector_setting['feature_classifier_id'])

In [41]:
win_id = detector_setting['windowing_id']

In [42]:
motor_nuclei = ['Amb', '3N', '4N', '5N', '6N', '7N', '10N', '12N']

In [43]:
# output_resolution = '10.0um'

In [67]:
registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection = defaultdict(dict)

section_margin_um = 200.
section_margin = 10
image_margin_um = 200.
image_margin = int(np.round(image_margin_um / convert_resolution_string_to_um('raw', stack)))

valid_secmin = np.min(metadata_cache['valid_sections'][stack])
valid_secmax = np.max(metadata_cache['valid_sections'][stack])

for name_u in all_known_structures:
# for name_u in motor_nuclei:
# for name_u in ['RtTg', 'VCA', 'AP', 'Tz', 'LC', 'Sp5O', '7n', 'SNC', 'Sp5I', 'DC', 'Sp5C', 'VCP', 'RMC', 'PBG', 'SNR', 'LRt', 'SC', 'IC', 'Pn', 'VLL']:
        
    if name_u in singular_structures:
        
        (xmin, ymin, secmin), (xmax, ymax, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[name_u]
        
        for sec in range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax)):
            
            if is_invalid(sec=sec, stack=stack):
                continue
                
            registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u][sec] = \
            (max(xmin - image_margin, 0), 
             xmax + image_margin, 
             max(ymin - image_margin, 0), 
             ymax + image_margin)
    else:
        
        a = defaultdict(list)
        
        lname = convert_to_left_name(name_u)        
        (xmin, ymin, secmin), (xmax, ymax, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[lname]
        
        for sec in range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax)):

            if is_invalid(sec=sec, stack=stack):
                continue
                
            a[sec].append((max(xmin - image_margin, 0), 
             xmax + image_margin, 
             max(ymin - image_margin, 0), 
             ymax + image_margin))
        
        rname = convert_to_right_name(name_u)
        (xmin, ymin, secmin), (xmax, ymax, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[rname]
    
        for sec in range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax)):

            if is_invalid(sec=sec, stack=stack):
                continue
                
            a[sec].append((max(xmin - image_margin, 0), 
             xmax + image_margin, 
             max(ymin - image_margin, 0), 
             ymax + image_margin))
            
        for sec, bboxes in a.iteritems():
            if len(bboxes) == 1:
                registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u][sec] = bboxes[0]
            else:
                xmin, ymin = np.min(bboxes, axis=0)[[0,2]]
                xmax, ymax = np.max(bboxes, axis=0)[[1,3]]
                registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u][sec] = (xmin, xmax, ymin, ymax)

In [101]:
# registered_atlas_structures_wrt_wholebrainXYcropped_roiSections = {}

# for name_s, ((xmin, ymin, secmin), (xmax, ymax, secmax)) in registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners.iteritems():
    
#     registered_atlas_structures_wrt_wholebrainXYcropped_roiSections[name_s] = \
#     [sec for sec in range(max(secmin - 10, metadata_cache['section_limits'][stack][0]), 
#                          min(secmin + 1 + 10, metadata_cache['section_limits'][stack][1]))
#                    if not is_invalid(sec=sec, stack=stack)]

In [86]:
# atlas_structures_wrt_wholebrainWithMargin_sections = defaultdict(list)
# for name_s, (s1, s2) in atlas_structures_wrt_wholebrainWithMargin_section_limits.iteritems():
#     atlas_structures_wrt_wholebrainWithMargin_sections[convert_to_original_name(name_s)] += range(max(s1 - 10, metadata_cache['section_limits'][stack][0]), 
#                                                                                                   min(s2+1 + 10, metadata_cache['section_limits'][stack][1]))

# atlas_structures_wrt_wholebrainWithMargin_sections = {name_s: sorted(set(sec_list)) for name_s, sec_list in atlas_structures_wrt_wholebrainWithMargin_sections.iteritems()}
# # atlas_structures_wrt_wholebrainWithMargin_sections.default_factory = None

In [ ]:
for name_u in all_known_structures:
# for name_u in motor_nuclei:
# for name_u in ['10N']:
# for name_u in ['RtTg', 'VCA', 'AP', 'Tz', 'LC', 'Sp5O', '7n', 'SNC', 'Sp5I', 'DC', 'Sp5C', 'VCP', 'RMC', 'PBG', 'SNR', 'LRt', 'SC', 'IC', 'Pn', 'VLL']:
            
    for sec, bbox in registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u].iteritems():
        
#         if is_invalid(sec=sec, stack=stack):
#             continue

        print name_u, sec
            
        viz_all_landmarks, scoremap_all_landmarks = \
        draw_scoremap(clfs={name_u: clfs[name_u]}, 
                              bbox=bbox,
                        scheme='none', 
                    win_id=win_id, prep_id=2,
                    stack=stack, return_scoremap=True, sec=sec,
                    model=model, model_name=model_name,
                     mean_img=mean_img, 
                     batch_size=batch_size,
                      output_patch_size=224,
                      is_nissl=False,
                   out_resolution_um=10.,
                image_shape=metadata_cache['image_shape'][stack],
                              return_wholeimage=True)
        
        sm = scoremap_all_landmarks[name_u]
        viz = viz_all_landmarks[name_u]
    
        scoremap_bp_filepath = \
        DataManager.get_downscaled_scoremap_filepath(stack=stack, section=sec, 
                                                     structure=name_u,
                                                     detector_id=detector_id,
                                                     out_resolution_um=out_resolution_um)
        save_data(sm.astype(np.float16), scoremap_bp_filepath, upload_s3=False)

        viz_filepath = \
        DataManager.get_scoremap_viz_filepath_v2(stack=stack, section=sec, 
                                                     structure=name_u,
                                                     detector_id=detector_id,
                                                     out_resolution=output_resolution)
        save_data(viz, viz_filepath, upload_s3=False)

        del viz_all_landmarks, scoremap_all_landmarks
#         break

5N 128


locate patches: 0.02 seconds
Load pre-computed features: 0.65 seconds
Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg/CHATM2_slide44_2018_01_22-S1_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg.jpg


Load background image: 2.49 seconds
Rescale background image to output resolution: 0.14 seconds
Predict scores 5N: 0.01 seconds
Resample scoremap 5N: 0.09 seconds
Load and rescale background image: 0.00 seconds
Generate scoremap overlay: 0.05 seconds.
Scoremap size does not match background image size. Need to resize: 0.08 seconds.
Generate scoremap overlay image 5N: 0.16 seconds
locate patches: 0.02 seconds


(597, 925, 3) (597, 925)
5N 129


...

Load background image: 2.78 seconds
Rescale background image to output resolution: 0.04 seconds
Predict scores 6N: 0.00 seconds
Resample scoremap 6N: 0.09 seconds
Load and rescale background image: 0.00 seconds
Generate scoremap overlay: 0.06 seconds.
Scoremap size does not match background image size. Need to resize: 0.08 seconds.
Generate scoremap overlay image 6N: 0.16 seconds
locate patches: 0.02 seconds


(597, 925, 3) (597, 925)
6N 168


Load pre-computed features: 0.59 seconds
Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg/CHATM2_slide56_2018_01_22-S3_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg.jpg


Load background image: 2.52 seconds
Rescale background image to output resolution: 0.04 seconds


In [33]:
# for stack in ['CHATM3']:
    
# registered_atlas_structures_wrt_wholebrainWithMargin_section_limits = load_json('/home/yuncong/' + stack + '_structure_section_limits.json')

#     atlas_structures_wrt_wholebrainWithMargin_sections = defaultdict(list)
#     for name_s, (s1, s2) in registered_atlas_structures_wrt_wholebrainWithMargin_section_limits.iteritems():
#         atlas_structures_wrt_wholebrainWithMargin_sections[convert_to_original_name(name_s)] += range(max(s1 - 10, metadata_cache['section_limits'][stack][0]), 
#                                                                                                       min(s2+1 + 10, metadata_cache['section_limits'][stack][1]))
#     atlas_structures_wrt_wholebrainWithMargin_sections = {name_s: sorted(set(sec_list)) for name_s, sec_list in atlas_structures_wrt_wholebrainWithMargin_sections.iteritems()}

# Specify isotropic resolution of the output volume.
out_resolution_um = convert_resolution_string_to_um(resolution=output_resolution, stack=stack)

# for structure in ['7N']:
for structure in motor_nuclei:


    for name_s in [convert_to_left_name(structure), convert_to_right_name(structure)]:
        
        scoremaps = {}

#         for sec in metadata_cache['valid_sections'][stack]:

        (xmin, ymin, s1), (xmax, ymax, s2) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[name_s]

        for sec in range(max(s1 - section_margin, metadata_cache['section_limits'][stack][0]), 
                         min(s2 + 1 + section_margin, metadata_cache['section_limits'][stack][1])):

            if is_invalid(sec=sec, stack=stack):
                continue

            scoremap = DataManager.load_downscaled_scoremap(stack=stack, section=sec, structure=structure, 
                                                            prep_id='alignedBrainstemCrop',
                                                          out_resolution_um=out_resolution_um, 
                                                            detector_id=detector_id).astype(np.float32)

            mask = DataManager.load_image_v2(stack=stack, section=sec, 
                                 prep_id='alignedBrainstemCrop', 
                                 resol='thumbnail', version='mask')

            mask_outResol = rescale_by_resampling(mask, new_shape=(scoremap.shape[1], scoremap.shape[0]))

            scoremap[~mask_outResol] = 0
            scoremaps[sec] = scoremap

        volume_outVolResol, volume_origin_wrt_wholebrainXYcropped_outVolResol = \
        images_to_volume_v2(images=scoremaps, spacing_um=20.,
                                in_resol_um=out_resolution_um,
                                out_resol_um=out_resolution_um)
        print volume_outVolResol.shape

        brain_spec = dict(name=stack,
                       vol_type='score',
                        detector_id=detector_id,
                       resolution=output_resolution)

        # Save volume and origin.

        save_data(volume_outVolResol.astype(np.float16), \
                  DataManager.get_original_volume_filepath_v2(stack_spec=brain_spec, structure=name_s))

        wholebrainXYcropped_origin_wrt_wholebrain_outVolResol = \
        DataManager.get_domain_origin(stack=stack, domain='wholebrainXYcropped', 
                                      resolution=output_resolution)
        volume_origin_wrt_wholebrain_outVolResol =\
        volume_origin_wrt_wholebrainXYcropped_outVolResol + wholebrainXYcropped_origin_wrt_wholebrain_outVolResol

        save_data(volume_origin_wrt_wholebrain_outVolResol,
                  DataManager.get_original_volume_origin_filepath_v3(stack_spec=brain_spec, structure=name_s, wrt='wholebrain'))

        # Compute gradients.

        gradients = compute_gradient_v2((volume_outVolResol, volume_origin_wrt_wholebrain_outVolResol), 
                                        smooth_first=True)

        DataManager.save_volume_gradients(gradients, stack_spec=brain_spec, structure=name_s)

Not using image_cache.
Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide36_2018_01_22-S1_prep2_thumbnail_mask.png
/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide35_2018_01_20-S2_prep2_thumbnail_mask.png


Not using image_cache.
Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide36_2018_01_22-S2_prep2_thumbnail_mask.png
/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide36_2018_01_22-S3_prep2_thumbnail_mask.png


Not using image_cache.
Not using image_cache.


...

/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide66_2018_02_14-S2_prep2_thumbnail_mask.png
/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide66_2018_02_14-S3_prep2_thumbnail_mask.png


Not using image_cache.
Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide68_2018_02_14-S1_prep2_thumbnail_mask.png
/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide68_2018_02_14-S2_prep2_thumbnail_mask.png


Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide70_2018_02_14-S1_prep2_thumbnail_mask.png
(590, 917, 125)
aws s3 cp "/home/yuncong/CSHL_volumes/CHATM2/CHATM2_detector799_10.0um_scoreVolume/score_volumes/CHATM2_detector799_10.0um_scoreVolume_12N.bp" "s3://mousebrainatlas-data/CSHL_volumes/CHATM2/CHATM2_detector799_10.0um_scoreVolume/score_volumes/CHATM2_detector799_10.0um_scoreVolume_12N.bp"
0
aws s3 cp "/home/yuncong/CSHL_volumes/CHATM2/CHATM2_detector799_10.0um_scoreVolume/score_volumes/CHATM2_detector799_10.0um_scoreVolume_12N_origin_wrt_wholebrain.txt" "s3://mousebrainatlas-data/CSHL_volumes/CHATM2/CHATM2_detector799_10.0um_scoreVolume/score_volumes/CHATM2_detector799_10.0um_scoreVolume_12N_origin_wrt_wholebrain.txt"
0
aws s3 cp "/home/yuncong/CSHL_volumes/CHATM2/CHATM2_detector799_10.0um_scoreVolume/score_volume_gradients/CHATM2_detector799_10.0um_scoreVolume_12N_gradients.bp" "s3://mousebrainatlas-data/CSHL_volumes/CHATM2/CHATM2_detector799_10.0um_scoreVol